<div>
    <div style="float:left;">
        <img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128" width="50%" />
    </div>
    <div style="float:left;">
        <img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif" width="50%"/>
    </div>
</div>

In [1]:
import ROOT
from ROOT import TFile, TMVA, TCut

Welcome to JupyROOT 6.07/07


# Import JsMVA and enable JS visualization

In [2]:
import sys, os
sys.path.append(os.path.expanduser("~/code/GSOC/wd/src/python"))
import JsMVA

In [3]:
%jsmva on

## Declare Factory and DataLoader

In [4]:
outputFile = TFile("files/TMVAOutput.root", "RECREATE")
inputFile  = TFile("files/mydataset.root")

TMVA.Tools.Instance()

factory = TMVA.Factory("TMVAClassification",
                       outputFile,
                       "!V:ROC:!Correlations:!Silent:Color:!DrawProgressBar:AnalysisType=Classification")
loader = TMVA.DataLoader("mydataset")

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


# General steps
## Adding variables to DataLoader

In [5]:
# Adding variables to dataset
loader.AddVariable("var0", 'F')
loader.AddVariable("var1", 'F')
loader.AddVariable("var2", 'F')
loader.AddVariable("var3 := var0-var1", 'F')
loader.AddVariable("var4 := var0*var2", 'F')
loader.AddVariable("var5 := var1+var2", 'F')

## Setting up dataset from Trees

In [6]:
print "--- TMVAClassification       : Using input file:", inputFile.GetName()
   
# Register the training and test trees

tsignal     = inputFile.Get("MyMCSig")
tbackground = inputFile.Get("MyMCBkg")
     
signalWeight     = 1.0
backgroundWeight = 1.0

mycuts = TCut("")
mycutb = TCut("")

loader.AddSignalTree(tsignal, signalWeight)
loader.AddBackgroundTree(tbackground, backgroundWeight)
loader.fSignalWeight = signalWeight
loader.fBackgroundWeight = backgroundWeight
loader.fTreeS = tsignal
loader.fTreeB = tbackground
loader.PrepareTrainingAndTestTree(mycuts,
                                  mycutb,
                                  "nTrain_Signal=3000:nTrain_Background=3000:nTest_Signal=1449:nTest_Background=1449:SplitMode=Random:NormMode=NumEvents:!V")

--- TMVAClassification       : Using input file: files/mydataset.root
--- DataSetInfo              : Dataset[mydataset] : Added class "Signal"	 with internal class number 0
--- mydataset                : Add Tree MyMCSig of type Signal with 5449 events
--- DataSetInfo              : Dataset[mydataset] : Added class "Background"	 with internal class number 1
--- mydataset                : Add Tree MyMCBkg of type Background with 5449 events
--- mydataset                : Preparing trees for training and testing...


# New method for visualizing input variables

# Input

In [7]:
loader.DrawInputVariable("var0")

--- DataSetFactory           : Dataset[mydataset] : Splitmode is: "RANDOM" the mixmode is: "SAMEASSPLITMODE"
--- DataSetFactory           : Dataset[mydataset] : Create training and testing trees -- looping over class "Signal" ...
--- DataSetFactory           : Dataset[mydataset] : Weight expression for class 'Signal': ""
--- DataSetFactory           : Dataset[mydataset] : Create training and testing trees -- looping over class "Background" ...
--- DataSetFactory           : Dataset[mydataset] : Weight expression for class 'Background': ""
--- DataSetFactory           : Dataset[mydataset] : Number of events in input trees (after possible flattening of arrays):
--- DataSetFactory           : Dataset[mydataset] :     Signal          -- number of events       : 5449   / sum of weights: 5449 
--- DataSetFactory           : Dataset[mydataset] :     Background      -- number of events       : 5449   / sum of weights: 5449 
--- DataSetFactory           : Dataset[mydataset] :     Signal     tre

## Transformed input

In [8]:
loader.DrawInputVariable("var0", processTrfs="D") #I;N;D;P;U;G,D

--- JsMVA.Factory            : Dataset[mydataset] : Create Transformation "D" with events from all classes.
--- Deco                     : Transformation, Variable selection : 
--- Deco                     : Input : variable 'var0' (index=0).   <---> Output : variable 'var0' (index=0).
--- Deco                     : Input : variable 'var1' (index=1).   <---> Output : variable 'var1' (index=1).
--- Deco                     : Input : variable 'var2' (index=2).   <---> Output : variable 'var2' (index=2).
--- Deco                     : Input : variable 'var3' (index=3).   <---> Output : variable 'var3' (index=3).
--- Deco                     : Input : variable 'var4' (index=4).   <---> Output : variable 'var4' (index=4).
--- Deco                     : Input : variable 'var5' (index=5).   <---> Output : variable 'var5' (index=5).
--- Deco                     : Preparing the Decorrelation transformation...
--- <WARNING> Tools                    : <GetSQRootMatrix> error in matrix diagonaliza

## Correlation matrix

In [9]:
loader.DrawCorrelationMatrix("Signal")

# General steps

## Booking classifiers

In [10]:
# Boosted Decision Trees
factory.BookMethod(loader,
                   TMVA.Types.kBDT,
                   "BDT",
                   "!V:NTrees=5:MinNodeSize=2.5%:MaxDepth=2:CreateMVAPdfs:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20")

# TMVA ANN: MLP Multilayer Perceptrons
factory.BookMethod(loader,
                   TMVA.Types.kMLP,
                   "MLP",
                   "!V:NeuronType=tanh:VarTransform=N:NCycles=150:CreateMVAPdfs:HiddenLayers=N+5:TestRate=5:!UseRegulator")

<ROOT.TMVA::MethodMLP object ("MLP") at 0x5af30d0>

--- Factory                  : Booking method: BDT DataSet Name: mydataset
--- Factory                  : Booking method: MLP DataSet Name: mydataset
--- MLP                      : Dataset[mydataset] : Create Transformation "N" with events from all classes.
--- Norm                     : Transformation, Variable selection : 
--- Norm                     : Input : variable 'var0' (index=0).   <---> Output : variable 'var0' (index=0).
--- Norm                     : Input : variable 'var1' (index=1).   <---> Output : variable 'var1' (index=1).
--- Norm                     : Input : variable 'var2' (index=2).   <---> Output : variable 'var2' (index=2).
--- Norm                     : Input : variable 'var3' (index=3).   <---> Output : variable 'var3' (index=3).
--- Norm                     : Input : variable 'var4' (index=4).   <---> Output : variable 'var4' (index=4).
--- Norm                     : Input : variable 'var5' (index=5).   <---> Output : variable 'var5' (index=5).
--- MLP      

In [17]:
layoutString = "Layout=TANH|100,TANH|50,TANH|10,LINEAR"

training0 = "LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,"
training0+= "WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True"
training1 = "LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,"
training1+= "WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1"
training2 = "LearningRate=1e-2,Momentum=0.3,Repetitions=1,ConvergenceSteps=300,BatchSize=40,TestRepetitions=7,"
training2+= "WeightDecay=0.0001,Regularization=L2,Multithreading=True"
training3 = "LearningRate=1e-3,Momentum=0.1,Repetitions=1,ConvergenceSteps=200,BatchSize=70,TestRepetitions=7,"
training3+= "WeightDecay=0.0001,Regularization=NONE,Multithreading=True"

trainingStrategyString = "TrainingStrategy="
trainingStrategyString += training0 + "|" + training1 + "|" + training2 + "|" + training3

nnOptions = "!H:V:VarTransform=Normalize:ErrorStrategy=CROSSENTROPY"
#nnOptions = "!H:V:ErrorStrategy=CROSSENTROPY:VarTransform=G:WeightInitialization=XAVIERUNIFORM"
#nnOptions = "!H:V:VarTransform=Normalize:ErrorStrategy=CHECKGRADIENTS"

nnOptions += ":" 
nnOptions += layoutString
nnOptions += ":"
nnOptions += trainingStrategyString

#factory.BookMethod(loader, TMVA.Types.kDNN, "DNN", nnOptions )

<ROOT.TMVA::MethodDNN object ("DNN") at 0x8537d20>

--- Factory                  : Booking method: DNN DataSet Name: mydataset
--- Configurable             : Parsing option string: 
--- Configurable             : ... "!H:V:VarTransform=Normalize:ErrorStrategy=CROSSENTROPY:Layout=TANH|100,TANH|50,TANH|10,LINEAR:TrainingStrategy=LearningRate=1e-1,Momentum=0.0,Repetitions=1,ConvergenceSteps=300,BatchSize=20,TestRepetitions=15,WeightDecay=0.001,Regularization=NONE,DropConfig=0.0+0.5+0.5+0.5,DropRepetitions=1,Multithreading=True|LearningRate=1e-2,Momentum=0.5,Repetitions=1,ConvergenceSteps=300,BatchSize=30,TestRepetitions=7,WeightDecay=0.001,Regularization=L2,Multithreading=True,DropConfig=0.0+0.1+0.1+0.1,DropRepetitions=1|LearningRate=1e-2,Momentum=0.3,Repetitions=1,ConvergenceSteps=300,BatchSize=40,TestRepetitions=7,WeightDecay=0.0001,Regularization=L2,Multithreading=True|LearningRate=1e-3,Momentum=0.1,Repetitions=1,ConvergenceSteps=200,BatchSize=70,TestRepetitions=7,WeightDecay=0.0001,Regularization=NONE,Multithreading=True"
--- Configura

## Training

In [11]:
factory.TrainAllMethods()

--- Factory                  :  
--- Factory                  : Train all methods for Classification ...
--- Factory                  : 
--- Factory                  : current transformation string: 'I'
--- Factory                  : Dataset[mydataset] : Create Transformation "I" with events from all classes.
--- Id                       : Transformation, Variable selection : 
--- Id                       : Input : variable 'var0' (index=0).   <---> Output : variable 'var0' (index=0).
--- Id                       : Input : variable 'var1' (index=1).   <---> Output : variable 'var1' (index=1).
--- Id                       : Input : variable 'var2' (index=2).   <---> Output : variable 'var2' (index=2).
--- Id                       : Input : variable 'var3' (index=3).   <---> Output : variable 'var3' (index=3).
--- Id                       : Input : variable 'var4' (index=4).   <---> Output : variable 'var4' (index=4).
--- Id                       : Input : variable 'var5' (index=5).   <-

## Evaluation

In [12]:
factory.TestAllMethods()

factory.EvaluateAllMethods()

--- Factory                  : Test all methods...
--- Factory                  : Test method: BDT for Classification performance
--- BDT                      : Dataset[mydataset] : Evaluation of BDT on testing sample (2898 events)
--- BDT                      : Dataset[mydataset] : Elapsed time for evaluation of 2898 events: 0.00125 sec       
--- BDT                      : Dataset[mydataset] : Evaluation of BDT on testing sample
--- BDT                      : Dataset[mydataset] : Elapsed time for evaluation of 2898 events: 0.00136 sec       
--- Factory                  : Test method: MLP for Classification performance
--- MLP                      : Dataset[mydataset] : Evaluation of MLP on testing sample (2898 events)
--- MLP                      : Dataset[mydataset] : Elapsed time for evaluation of 2898 events: 0.00673 sec       
--- MLP                      : Dataset[mydataset] : Evaluation of MLP on testing sample
--- MLP                      : Dataset[mydataset] : Elapsed time f

# Classifier output distribution

In [13]:
factory.DrawOutputDistribution("mydataset", "MLP")

# Classifier Probability Distribution


In [14]:
factory.DrawProbabilityDistribution("mydataset", "MLP")

# ROC curve

In [15]:
factory.DrawROCCurve("mydataset")

# Classifier cut efficiencies

In [16]:
factory.DrawCutEfficiencies("mydataset", "BDT")

# Neural Network

* Mouseover (node, weight): focusing
* Zooming and grab and move supported
* Reset: double click

In [21]:
factory.DrawNeuralNetwork("mydataset", "MLP")

# Deep Neural network

In [19]:
factory.DrawNeuralNetwork("mydataset", "DNN")

# Decision Tree

* Mouseover (node, weight): showing decision path
* Zooming and grab and move supported
* Reset: double click
* Click on node: 
    * hiding subtree, if node children are hidden the node will have a green border
    * rescaling: bigger nodes, bigger texts
    * click again to show the subtree

In [20]:
factory.DrawDecisionTree("mydataset", "BDT")

In [22]:
outputFile.Close()